<img width="8%" alt="GitHub.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/GitHub.png" style="border-radius: 15%">

# GitHub - Connect from Naas Chat
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=GitHub+-+Connect+from+Naas+Chat:+Error+short+description">Bug report</a>

**Tags:** #github #connect #naas #secret #command #chat

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel)

**Last update:** 2023-09-29 (Created: 2023-09-29)

**Description:** This notebook integrates your GitHub token into the Naas secret manager under the variable 'GITHUB_TOKEN' from Naas Chat and connect to your GitHub organization.

**References:**
- [Naas Secret Documentation](https://site.naas.ai/docs/developers/jobs/secret)
- [Generate a personal access token](https://docs.github.com/en/github/authenticating-to-github/creating-a-personal-access-token)
- [Go to personal access token](https://github.com/settings/tokens)

## Input

### Import libraries

In [1]:
import naas
import requests
import os

### Setup variables
- `github_token`: GitHub token to be add as secret.
- `github_url`: GitHub organization html URL.
- `body`: This variable stores the body to be send by the webhook.

In [2]:
github_token = None
github_url = "https://github.com/jupyter-naas/"
body = {}

### Setup parameters
The webhook body will be injected below this cell when the webhook is triggered. 
Therefore, it is important to set up how you will handle the injected variable from the body in order to make your script work.
To receive the body from the webhook, please ensure that this cell is tagged as "parameters".

In [3]:
# Parameters
if len(body) > 0:
    github_token = body.get("github_token")
    github_url = body.get("github_url")

## Model

### Connect to GitHub

In [8]:
def add_or_update_secret(
    secret_name,
    secret_value
):
    # Init
    value = naas.secret.get(secret_name)
    
    # Add secret
    if secret_value and value != secret_value:
        value = secret_value
    return value

def get_github_org(
    token,
    html_url
):
    # Init
    data = {}
    base_url = "https://api.github.com/orgs/"
    
    # Split URL
    org = html_url.split("https://github.com/")[-1].split("/")[0]
    
    # Create URL
    url = os.path.join(base_url, org)
    
    # Set the request headers
    headers = {
        "Authorization": f"token {token}",
        "Content-Type": "application/json"
    }
    
    # Send the POST request
    res = requests.get(url, headers=headers)
    if res.status_code == 200:
        data = res.json()
    return data

def connect_github(
    secret_name,
    secret_value,
    github_url,
):
    # Init
    status = "ok"
    try:
        # Add GitHub token to naas secret
        token = add_or_update_secret(secret_name, secret_value)
        
        # Connect to GitHub org
        data = get_github_org(token, github_url)
        if len(data):
            org_name = data.get("login")
            description = data.get("description")
            public_repos = data.get("public_repos")
            total_private_repos = data.get("total_private_repos")
            github_org = add_or_update_secret("GITHUB_ORG", org_name)
            message = f"Connexion to GitHub successfull. Organization: {org_name}, Description: {description}, Public repos: {public_repos}, Private repos: {total_private_repos}"
        else:
            status = "ko"
            message = "Impossible to retrieve data from your organization please retry to connect!"
    except Exception as e:
            status = "ko"
            message = f"Template error: {e}"
    return status, message

status, message = connect_github("GITHUB_TOKEN", github_token, github_url)

## Output

### Return JSON response
Response sent to the browser before displayed in Chat UI.

In [9]:
naas.webhook.respond_json(
    {
        "status": status, 
        "message": message
    }
)